# Problem 1: TensorFlow and Edge Detection using SVM

In [5]:
%matplotlib inline
# from __future__ import division
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.compat.v2 as tf
import shutil
from utils import *
from HOG import *
from svm_tf import *

In [7]:
datasets = np.load('pedestrian_dataset.npz')    # extracted from the original Dalal and Triggs paper dataset available here: http://pascal.inrialpes.fr/data/human/
[k for k in datasets.keys()]

['eval_neg', 'train_neg', 'test_pos', 'eval_pos', 'train_pos', 'test_neg']

In [8]:
### EXPLORE THE DATASET!
# p=0 are negative examples
# p=1 are positive examples
@interact(k = (0, datasets["train_neg"].shape[0] - 1), p = (0, 1))
def view_img(k, p):
    d = {0: "train_neg", 1: "train_pos"}
    plt.imshow(datasets[d[p]][k])

interactive(children=(IntSlider(value=249, description='k', max=499), IntSlider(value=0, description='p', max=…

In [9]:
### VISUALIZE HOGs!
# p=0 are negative examples
# p=1 are positive examples
@interact(k = (0, datasets["train_neg"].shape[0]-1), p = (0, 1))
def view_img(k, p):
    d = {0: "train_neg", 1: "train_pos"}
    plt.figure(figsize = (15,15))    # feel free to change this depending on your screen resolution
    plt.imshow(datasets[d[p]][k])
    plot_cell_hogs(tf_histogram_of_oriented_gradients(datasets[d[p]][k])[0])

interactive(children=(IntSlider(value=249, description='k', max=499), IntSlider(value=0, description='p', max=…

# Do this after you have trained your SVM with HOG features

In [10]:
### VISUALIZE WEIGHTED HOGs!
##################################
(x_train, y_train), (x_eval, y_eval), (x_pred, y_true) = get_hog_data()
data = {}
data['x_train'] = x_train
data['y_train'] = y_train
data['x_eval'] = x_eval
data['y_eval'] = y_eval
data['x_pred'] = x_pred
data['y_true']  = y_true
data['name'] = 'hog'

model_dir = "hog_model/trained_weights"

dim = data['x_train'].shape[-1]
svm_model = SVM(dim=dim)
svm_model.load_weights(model_dir)

data = np.concatenate([datasets["test_pos"], datasets["test_neg"]], axis=0)

weights = np.reshape(svm_model.W.numpy(), [1, -1])
pos_weights = np.maximum(weights, 0)
neg_weights = -np.minimum(weights, 0)
file_exists = False
try:
    misclass_idx = np.load("hog_misclass_idx.npy")
    file_exists = True
except OSError:
    print("\n\nYou need to run the svm with HOG features first! hog_misclass_idx.py doesn't exist yet.\n\n")

if file_exists:
    @interact(k = (0, data.shape[0]-1), show_image = (False, True))
    def view_img(k, show_image):

        misclass_idx = np.load("hog_misclass_idx.npy")
        plt.figure(figsize = (15,15))    # feel free to change this depending on your screen resolution
        plt.imshow(data[k])
        hog = hog_descriptor(data[k]).numpy()
        unweighted_block_hogs = np.mean(hog.reshape([16, 8, 1, 9]), axis=2)
        pos_weighted_block_hogs = np.mean(pos_weights.reshape([16, 8, 1, 9])*hog.reshape([16, 8, 1, 9]), axis=2)
        neg_weighted_block_hogs = np.mean(neg_weights.reshape([16, 8, 1, 9])*hog.reshape([16, 8, 1, 9]), axis=2)

        plt.subplot(1,3,1)
        color = 'yellow' if show_image else 'white'
        plt.imshow(show_image*data[k])
        plot_cell_hogs(unweighted_block_hogs, pixels_in_cell=8, color=color)
        if k in misclass_idx:
            plt.title("Misclassified!")
        else:
            plt.title("Classified correctly")

        plt.subplot(1,3,2)
        plt.imshow(show_image*data[k])
        plot_cell_hogs(pos_weighted_block_hogs, pixels_in_cell=8, color=color)
        plt.title("Positive Weights")

        plt.subplot(1,3,3)
        plt.imshow(show_image*data[k])
        plot_cell_hogs(neg_weighted_block_hogs, pixels_in_cell=8, color=color)
        plt.title("Negative Weights")

InvalidArgumentError: Unsuccessful TensorSliceReader constructor: Failed to get matching files on hog_model/trained_weights: Not found: hog_model; No such file or directory